In [ ]:
pip install tensorflow


In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [43]:
import pandas as pd
from sklearn.utils import resample

In [14]:
df = pd.read_csv('/sentiment.csv',header=None)


In [19]:
df.columns=['col1','col2','sentiment','summary']

In [20]:
df.head()

,col1,col2,sentiment,summary
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [23]:
df=df[['sentiment','summary']]
df.head()

,sentiment,summary
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...


In [37]:
df.to_csv('new_sentiment.csv',index=False)
dfs=pd.read_csv('new_sentiment.csv')
dfs.head()


,sentiment,summary
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...


In [34]:
df=dfs[~dfs['sentiment'].isin(['Irrelevant', 'Neutral'])]
df.shape

(43374, 2)

In [35]:
df.sentiment.value_counts()

sentiment
Negative    22542
Positive    20832
Name: count, dtype: int64

In [40]:
sentiment_mapping= {'Positive': 1, 'Negative': 0}
df['sentiment'] = df['sentiment'].map(sentiment_mapping)
df.head(10)

,sentiment,summary
0,1,im getting on borderlands and i will murder yo...
1,1,I am coming to the borders and I will kill you...
2,1,im getting on borderlands and i will kill you ...
3,1,im coming on borderlands and i will murder you...
4,1,im getting on borderlands 2 and i will murder ...
5,1,im getting into borderlands and i can murder y...
6,1,So I spent a few hours making something for fu...
7,1,So I spent a couple of hours doing something f...
8,1,So I spent a few hours doing something for fun...
9,1,So I spent a few hours making something for fu...


In [45]:
# Separate the majority and minority classes
df_positive = df[df['sentiment'] == 1]
df_negative = df[df['sentiment'] == 0]
# Undersample the majority class (Negative)
df_negative_undersampled = resample(df_negative,
                                    replace=False,    # sample without replacement
                                    n_samples=len(df_positive), # to match the number of positive samples
                                    random_state=42) # for reproducibility

# Combine the minority class with the undersampled majority class
df_balanced = pd.concat([df_positive, df_negative_undersampled])

# Shuffle the dataset to mix the positive and negative samples
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)


In [52]:
df=df_balanced
df.head(10)
df.shape
df.sentiment.value_counts()

sentiment
1    20832
0    20832
Name: count, dtype: int64

In [73]:
import re

def preprocess_text(input_text, lower=True, split=" ", filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'):
    """
    Preprocesses input text by applying the following steps:
    1. Convert text to lowercase if specified.
    2. Remove specified characters (filters).
    3. Remove digits.
    4. Split text into tokens based on the specified split character.

    Parameters:
    input_text (str): The input text to preprocess.
    lower (bool): Whether to convert the text to lowercase (default is True).
    split (str): The character used for splitting the text into tokens (default is " ").
    filters (str): The characters to remove from the text (default is specified punctuation and whitespace characters).

    Returns:
    str: The preprocessed text.
    """
    # Print the type of input_text for debugging
    print("Type of input_text:", type(input_text))

    # Check if input_text is a string
    if isinstance(input_text, str):
        # Convert text to lowercase if specified
        if lower:
            input_text = input_text.lower()

        # Remove specified characters
        translate_dict = {c: split for c in filters}
        translated_text = input_text.translate(str.maketrans(translate_dict))

        # Remove digits
        translated_text = re.sub(r'\d+', '', translated_text)

    else:
        # If input_text is not a string, return it unchanged
        translated_text = input_text

    return translated_text


In [74]:
df['summary']=df['summary'].apply(preprocess_text)

Streaming output truncated to the last 5000 lines.
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str'>
Type of input_text: <class 'str

In [75]:
x=df['summary']
y=df['sentiment']

In [77]:
tokenizer=Tokenizer(num_words=10000,oov_token='xxxxxxx')
x = [str(text) for text in x]
tokenizer.fit_on_texts(x)
x_dict=tokenizer.word_index
len(x_dict)

21418

In [78]:
x_dict.items()

dict_items([('xxxxxxx', 1), ('the', 2), ('i', 3), ('to', 4), ('and', 5), ('a', 6), ('is', 7), ('of', 8), ('this', 9), ('it', 10), ('for', 11), ('in', 12), ('my', 13), ('you', 14), ('on', 15), ('that', 16), ('game', 17), ('so', 18), ('with', 19), ('me', 20), ('have', 21), ('not', 22), ('just', 23), ('but', 24), ('are', 25), ('be', 26), ('all', 27), ('was', 28), ('its', 29), ('im', 30), ('like', 31), ('at', 32), ('get', 33), ('now', 34), ('your', 35), ('from', 36), ('out', 37), ('they', 38), ('really', 39), ('play', 40), ('can', 41), ('good', 42), ('we', 43), ('as', 44), ('do', 45), ('love', 46), ('no', 47), ('one', 48), ('cant', 49), ('about', 50), ('when', 51), ('why', 52), ('new', 53), ('what', 54), ('if', 55), ('an', 56), ('will', 57), ('time', 58), ('shit', 59), ('how', 60), ('up', 61), ('has', 62), ('more', 63), ('still', 64), ('been', 65), ('or', 66), ('some', 67), ('dont', 68), ('games', 69), ('fuck', 70), ('playing', 71), ('fucking', 72), ('people', 73), ('got', 74), ('by', 75),

In [79]:
x_sequences=tokenizer.texts_to_sequences(x)
x_sequences[:10]

[[2583, 205, 20, 6, 842, 12, 126, 209, 7, 2360],
 [208, 2156, 397, 743, 30, 4021, 11, 324, 3455],
 [553,
  181,
  3,
  83,
  488,
  32,
  6,
  17,
  682,
  1244,
  3,
  46,
  511,
  682,
  5,
  182,
  3,
  83,
  488,
  302,
  15,
  6,
  511,
  11,
  182,
  180,
  1091,
  15,
  2,
  653,
  8,
  6193,
  13,
  5737,
  11,
  9,
  511,
  2843],
 [3, 547, 1616, 2494, 955, 1123, 51, 3, 28, 2759],
 [6],
 [193,
  287,
  283,
  251,
  7,
  458,
  6,
  699,
  177,
  451,
  36,
  675,
  1286,
  766,
  1,
  119,
  47,
  1360,
  58,
  785,
  66,
  4291,
  10,
  23,
  386,
  18,
  4613,
  2,
  1,
  7,
  697,
  2,
  545,
  25,
  256,
  3633,
  59,
  5,
  1501,
  12,
  1],
 [138, 158, 129, 9496, 29, 3064, 400, 11, 3170, 2844, 12, 3456, 832],
 [557, 12, 380, 7, 2018, 3822, 1971],
 [3,
  867,
  55,
  126,
  205,
  1732,
  5331,
  7647,
  471,
  26,
  6,
  3634,
  100,
  51,
  38,
  64,
  2760,
  61],
 [262, 7648, 395, 4022, 2, 820, 8, 2, 197, 37, 5, 21, 2219]]

In [82]:
x_padded_seq=pad_sequences(x_sequences,padding='post',maxlen=100)
x_padded_seq[:3]

array([[2583,  205,   20,    6,  842,   12,  126,  209,    7, 2360,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0],
       [ 208, 2156,  397,  743,   30, 4021,   11,  324, 3455,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    

In [83]:
x_padded_seq.shape

(41664, 100)

In [84]:
import numpy as np

In [85]:
y=np.array(y)
y=y.flatten()
max_length=100
vocab_size=10000
embedding_dims=50

In [95]:
import tensorflow as tf
model=tf.keras.Sequential([tf.keras.layers.Embedding(input_length=100,input_dim=10000,output_dim=50),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(50,activation='relu'),
tf.keras.layers.Dense(1,activation='sigmoid')])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 50)           500000    
                                                                 
 flatten_5 (Flatten)         (None, 5000)              0         
                                                                 
 dense_6 (Dense)             (None, 50)                250050    
                                                                 
 dense_7 (Dense)             (None, 1)                 51        
                                                                 
Total params: 750101 (2.86 MB)
Trainable params: 750101 (2.86 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [97]:
num_epochs=10
model.fit
 (x_padded_seq,y,epochs=num_epochs)

Epoch 1/10
1302/1302 [==============================] - 16s 12ms/step - loss: 0.3329 - accuracy: 0.8429
Epoch 2/10
1302/1302 [==============================] - 13s 10ms/step - loss: 0.1068 - accuracy: 0.9534
Epoch 3/10
1302/1302 [==============================] - 15s 11ms/step - loss: 0.0639 - accuracy: 0.9704
Epoch 4/10
1302/1302 [==============================] - 13s 10ms/step - loss: 0.0527 - accuracy: 0.9747
Epoch 5/10
1302/1302 [==============================] - 13s 10ms/step - loss: 0.0474 - accuracy: 0.9760
Epoch 6/10
1302/1302 [==============================] - 13s 10ms/step - loss: 0.0451 - accuracy: 0.9756
Epoch 7/10
1302/1302 [==============================] - 13s 10ms/step - loss: 0.0424 - accuracy: 0.9773
Epoch 8/10
1302/1302 [==============================] - 13s 10ms/step - loss: 0.0415 - accuracy: 0.9776
Epoch 9/10
1302/1302 [==============================] - 13s 10ms/step - loss: 0.0392 - accuracy: 0.9778
Epoch 10/10
1302/1302 [==============================] - 13s 10m

In [139]:
test=input("Enter your review::")
print(test)
test_sequences = tokenizer.texts_to_sequences(test)
test_padded = pad_sequences(test_sequences, maxlen=100, dtype='int32', value=0)
# print(test_padded.shape)
# print(test_padded)
predictions = model.predict(test_padded)[0]
if predictions[0] < 0.5:
    print("Negative")
else:
    print("Positive")
print(predictions)

Enter your review::worst movie i ever watched
worst movie i ever watched
1/1 [==============================] - 0s 20ms/step
Negative
[0.3112987]
